# Import Required Libraries

In [4]:
# Basic packages
import os
import sys
from typing import List, Dict
import warnings
warnings.filterwarnings("ignore")

# Standard libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# tensorflow packages
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import (Dense, 
                                     Input)
from tensorflow.keras.models import Model
                        

2.19.0


In [ ]:
afd